In [1]:
from utils.transformations import center_to_topleft, apply_transformation, get_matrices_for_smooth_transformation
import numpy as np

In [ ]:
v = np.array(((0, 1), (1, 1), (1, 0), (0, 0)))
m = np.array(((2, 0), (0, 1)))


[[400. 320.]
 [480. 320.]
 [480. 400.]
 [400. 400.]]
[[0 1]
 [2 1]
 [2 0]
 [0 0]]


array([[[1.  , 0.  ],
        [0.  , 1.  ]],

       [[1.25, 0.  ],
        [0.  , 1.  ]],

       [[1.5 , 0.  ],
        [0.  , 1.  ]],

       [[1.75, 0.  ],
        [0.  , 1.  ]],

       [[2.  , 0.  ],
        [0.  , 1.  ]]])

In [3]:
m = np.array([[1, 1], [0, 1]])  # Shear transformation matrix
steps = 5
smooth_matrices = get_matrices_for_smooth_transformation(m, steps)

In [3]:
k = ((1, 1))
m = np.array(((0, 2), (2, 0)))
smooth_matrices = get_matrices_for_smooth_transformation(m, 5)
print(apply_transformation(k, m))
k_temp = k
for M in smooth_matrices:
    print(apply_transformation(k, M))


[2 2]
[1. 1.]
[1.25 1.25]
[1.5 1.5]
[1.75 1.75]
[2. 2.]


In [ ]:
def smooth_transformation(m, steps=100, identity_m = np.array(((1, 0), (0, 1)))):
    m = np.array(m)
    transformations = np.linspace(identity_m, m, steps)
    return transformations

In [ ]:
def move_vec(s, e, steps=100, x_than_y=False):
    s = np.array(s)
    e = np.array(e)
    movement = np.linspace(s, e, steps)
    if x_than_y:
        movement = np.concat((np.linspace(s, np.array((e[0], s[1])), int(steps/2)), 
                              np.linspace(np.array((e[0], s[1])), e, int(steps/2))))
    return movement

In [ ]:
def draw_vec(screen, v, color=BLACK):
    pygame.draw.line(screen, color, center_to_topleft((0, 0)), center_to_topleft(v), 5)

In [ ]:
class Grid():
    def __init__(self, low=-10, high=10, step=1):
        self.generate(low, high, step)
    
    def generate(self, low, high, step):
        n = abs(int((low - high) / step)) + 1
        points = np.array(np.meshgrid(np.linspace(low, high, n), np.linspace(low, high, n)))
        x_axis = points.T[n//2]
        y_axis = points[:, n//2].T
        self.x_axis = x_axis
        self.y_axis = y_axis
        self.points = points
        self.points_transformed = points
        self.x_axis_transformed = x_axis
        self.y_axis_transformed = y_axis
    
    def transform(self, m):
        self.points_transformed = transform_vec(self.points.T.reshape(-1, 2), m).T.reshape(self.points.shape)
        self.x_axis_transformed = transform_vec(self.x_axis, m)
        self.y_axis_transformed = transform_vec(self.y_axis, m)
    
    def smooth_transformation(self, m, steps=100, identity_m = np.array(((1, 0), (0, 1)))):
        transformations = np.linspace(identity_m, m, steps)
        for t in transformations:
            self.transform(t)
            yield True

        
    
    def draw(self, screen, axis_color=BLUE, grid_color=WHITE, transformed = False, axis_lw=4, grid_lw=1):
        points = self.points_transformed.copy() if transformed else self.points.copy()
        x_axis = self.x_axis_transformed.copy() if transformed else self.x_axis.copy()
        y_axis = self.y_axis_transformed.copy() if transformed else self.y_axis.copy()
        for i in range(points.shape[1]):
            row = points.T[i]
            col = points[:, i].T
            if np.all(row == self.x_axis) or np.all(col == self.y_axis):
                continue
            pygame.draw.lines(screen, grid_color, False, center_to_topleft(row), grid_lw)
            pygame.draw.lines(screen, grid_color, False, center_to_topleft(col), grid_lw)

        # plot the origin axes
        pygame.draw.lines(screen, axis_color, False, center_to_topleft(x_axis), axis_lw)
        pygame.draw.lines(screen, axis_color, False, center_to_topleft(y_axis), axis_lw)

In [ ]:
x

array([[-1.42383345e+00,  2.67687901e+00],
       [-7.12655754e-01,  2.10258790e-01],
       [-6.78605762e-01,  5.64633650e-01],
       [ 1.07302764e+00, -1.62794188e+00],
       [ 3.88282328e-01,  9.14157744e-01],
       [ 6.63771803e-01,  4.77581270e-01],
       [ 7.76101356e-01,  3.68565109e-01],
       [-3.94429887e-01, -6.70001777e-02],
       [ 1.83974861e-01, -1.77261951e-02],
       [ 8.88510741e-01, -1.27298362e+00],
       [ 7.21698361e-01,  1.42750753e+00],
       [-1.25925378e+00, -2.93538108e-01],
       [-9.80864385e-01, -6.78834775e-01],
       [ 7.05203910e-01,  2.94348303e-01],
       [-2.52152766e-01, -8.58612393e-01],
       [-6.64534386e-01,  1.72631457e+00],
       [ 2.86645994e-02, -1.18808399e+00],
       [-7.78423444e-01,  1.43494495e+00],
       [-3.23533215e-01,  1.53410040e-01],
       [-9.99987347e-01, -1.18502616e+00],
       [ 1.20232486e-01,  2.23911841e-01],
       [ 1.75225051e-01,  1.38012007e+00],
       [ 1.12137644e+00,  1.83815867e-01],
       [-3.

In [ ]:
class Vector:

    def __init__(self, v, color=RED):
        self.v = v
        self.v_transformed = v
        self.color = color
    
    def draw(self, screen, transformed=False):
        v = self.v_transformed.copy() if transformed else self.v.copy()
        draw_vec(screen, v, self.color)
    
    def transform(self, m):
        self.v_transformed = transform_vec(self.v, m)

In [ ]:
grid = Grid(-10, 10, 1)
grid_base = Grid(-10, 10, 1)

In [ ]:
m = np.array(((1, 1), (-1, 1)))

In [ ]:
identity_m = np.array(((1, 0), (0, 1)))

In [ ]:
shear = np.array(((1, 1), (0, 1)))
rotate_45 = np.array(((0, 1), (-1, 0)))

In [ ]:
i_hat = np.array((1, 0))
j_hat = np.array((0, 1))

In [ ]:
transformations = smooth_transformation(rotate_45, steps=10, identity_m=identity_m)
transformations

In [ ]:
grid = Grid(-10, 10, 1)
grid_base = Grid(-10, 10, 1)

i_hat = Vector(np.array((1, 0)), color=GREEN)
j_hat = Vector(np.array((0, 1)), color=RED)

v = Vector(np.array((1, 1)), color=ORANGE)

m = np.array(((1, 1), (-2, 1)))

pygame.init()

# Set up the drawing window
screen = pygame.display.set_mode([WIDTH, HEIGHT])

# Run until the user asks to quit
running = True
first = True
while running:

    # Did the user click the window close button?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False    

    screen.fill(BLACK)
    grid_base.draw(screen, grid_color=GREY, axis_color=GREY, axis_lw=1, grid_lw=1)
    grid.draw(screen, grid_lw=2, transformed=True)
    i_hat.draw(screen, transformed=True)
    j_hat.draw(screen, transformed=True)
    v.draw(screen, transformed=True)
    transformations = smooth_transformation(m, steps=200)
    if first:
        for t in transformations:
            # Refresh the screen
            screen.fill(BLACK)
            grid_base.draw(screen, grid_color=GREY, axis_color=GREY, axis_lw=1, grid_lw=1)

            # transform all objects
            grid.transform(t)
            i_hat.transform(t)
            j_hat.transform(t)
            v.transform(t)

            # Draw all objects
            grid.draw(screen, grid_lw=2, transformed=True)
            i_hat.draw(screen, transformed=True)
            j_hat.draw(screen, transformed=True)
            v.draw(screen, transformed=True)

            # Flip
            pygame.display.flip()
            pygame.time.wait(10)
        first = False

    # for i, j in zip(smooth_transformation(i_hat, rotate_45), smooth_transformation(j_hat, rotate_45)):
    #     screen.fill(BLACK)
    #     grid_base.draw(screen, grid_color=GREY, axis_color=GREY, axis_lw=1, grid_lw=1)
    #     grid.draw(screen, grid_lw=2)

    #     draw_vec(screen, i, color=GREEN)
    #     draw_vec(screen, j, color=RED)
    #     # pygame.draw.circle(screen, RED, center_to_topleft(m), 5)
    #     pygame.display.flip()
    #     pygame.time.wait(10)
    # if first:
    #     grid.transform(m)
    #     first = False
    
    pygame.display.flip() 


In [ ]:
grid = Grid(-10, 10, 1)

In [ ]:
def draw_xy_axis(surface):
    lw = 4
    pygame.draw.line(surface, WHITE, (0, HEIGHT/2), (WIDTH, HEIGHT/2), lw)
    pygame.draw.line(surface, WHITE, (WIDTH/2, 0), (WIDTH/2, HEIGHT), lw)

def draw_grid(surface):
    lw = 1
    for i in range(-10, 11):
        pygame.draw.line(surface, WHITE, topleft2centerorigin((i, -10)), topleft2centerorigin((i, 10)), lw)
        pygame.draw.line(surface, WHITE, topleft2centerorigin((-10, i)), topleft2centerorigin((10, i)), lw)

In [ ]:
def connect_points(surface, points, color = WHITE, lw = 1):
    for i in range(len(points) - 1):
        pygame.draw.line(surface, color, topleft2centerorigin(points[i]), topleft2centerorigin(points[i + 1]), lw)

In [ ]:
def get_grid(n = 10):
    grid = []
    for i in range(-n, n+1):
        start = (i, -n)
        for j in range(-n, n+1):
            end = (i, j)
            grid.append((start, end))
            grid.append((i, j))
    return grid


    

In [ ]:
def draw_vec(surface, v, color = RED, origin = (0,0)):
    lw = 4
    origin = topleft2centerorigin(origin)
    v = topleft2centerorigin(v)
    pygame.draw.line(surface, color, origin, v, lw)
    pygame.draw.circle(surface, color, v, lw*2)


In [ ]:
def transform_vec(vec, matrix):
    return np.dot(matrix, vec)

In [ ]:
i_hat = np.array((1, 0))
j_hat = np.array((0, 1))

In [ ]:
rotate_45 = np.array(((0, -1), (1, 0)))
sheer = np.array(((1, 1), (0, 1)))

In [ ]:
def find_transition_points(start_v = (1, 0), end_v = (0, 1), steps = 100):
    """Find transition points between two vectors, such that the length of the vector is the same at each point"""
    v_len = np.sqrt(start_v[0]**2 + start_v[1]**2)
    # find points between start_v and end_v where the length of the vector is the same
    points_x = np.arange(start_v[0], end_v[0], (end_v[0] - start_v[0]) / steps)
    points_y = []
    for x in points_x:
        points_y.append(np.sqrt(v_len**2 - x**2))
    points = list(zip(points_x, points_y))
    
    return points

In [ ]:
transform_vec((300, 10000), rotate_45), transform_vec((300, -1000), rotate_45)

In [ ]:
def get_grid_matrix():
    return np.array(((1, 0), (0, 1)))

In [ ]:
points = np.linspace((0, 0), (0, 5), 5)
points

In [ ]:
pygame.init()

# Set up the drawing window
screen = pygame.display.set_mode([WIDTH, HEIGHT])

# Run until the user asks to quit
running = True
while running:

    # Did the user click the window close button?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the background with black
    screen.fill(BLACK)

    # Draw line
    draw_xy_axis(screen)
    draw_grid(screen)
    draw_vec(screen, i_hat, RED)
    draw_vec(screen, j_hat, GREEN)
    pygame.draw.line(screen, YELLOW, (300, HEIGHT), (300, 0), 4)
    pygame.draw.line(screen, YELLOW, transform_vec((300, 10000), sheer), transform_vec((300, -1000), sheer), 4)    # transition_ihat = find_transition_points(i_hat, transform_vec(i_hat, rotate_45))
    # transiation_jhat = find_transition_points(j_hat, transform_vec(j_hat, rotate_45))
    # for i, j in zip(transition_ihat, transiation_jhat):
    #     draw_vec(screen, i, RED)
    #     draw_vec(screen, j, GREEN)
    #     pygame.display.flip()
    #     pygame.time.delay(20)
    #     # refresh the screen
    #     screen.fill(BLACK)
    #     draw_xy_axis(screen)
    #     draw_grid(screen)
    
    # Flip the display
    pygame.display.flip()